To test that linear response theory is recovering the true amplification of freshwater fluxes, we need to evaluate the amplification of fluxes from CESM flux fields. Note that as linear response theory is finding the change in freshwater fluxes as a proportion of the FAFMIP perturbation, the truth is not exactly the change in magnitude of freshwater fluxes, but rather the change of the projection of the freshwater fluxes onto the FAFMIP pattern. There are other ways that we have thought of to quantify the true fluxes which are shown in the "true_freshwater_fluxes_options.ipynb" notebook.

In [1]:
import scipy.io
import netCDF4
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os, glob 
import imageio
from matplotlib import animation
import copy
import cartopy as cart
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter #see https://scitools.org.uk/cartopy/docs/v0.15/examples/tick_labels.html
import certifi
import ssl
import math
ssl._create_default_https_context = ssl._create_unverified_context
from scipy import stats
from xgcm import Grid
import statsmodels.api as sm
import matplotlib.ticker as ticker
from matplotlib.axes._secondary_axes import SecondaryAxis
import xesmf as xe

In [2]:
#First, load in all the freshwater fluxes. See notebook processing_salt_fluxes.py for how to generate this files that we unpickle here
import pickle 

with open("/scratch/abf376/regridded_salt_flux_2005on", "rb") as fp:   #Unpickling
    regridded_2005on=pickle.load(fp)

## First we calculate the true fluxes for the ensemble mean

In [3]:
#average over ensemble member
salt_flux_avg_2005on=sum(regridded_2005on)/34
salt_flux_avg_2005on=salt_flux_avg_2005on.rename({'y': 'latitude','x': 'longitude'})
salt_flux_avg_2005on=salt_flux_avg_2005on.assign_coords(latitude=salt_flux_avg_2005on.lat[:,0],longitude=salt_flux_avg_2005on.lon[0,:])

In [4]:
salt_flux_avg_2006to2055=salt_flux_avg_2005on[0:12*59,:,:] #salt_avg was from 1920 to 2080 so this is from 1970 to 2019

In [5]:
#create area grid

import sys
sys.path.insert(1, '/scratch/abf376/')
from area_grid import *

area=area_grid(latitudes=np.array(salt_flux_avg_2005on[0,:,:].latitude),longitudes=salt_flux_avg_2005on[0,:,:].longitude)
area=xr.DataArray(area,dims=["latitude","longitude"],coords=[salt_flux_avg_2005on[0,:,:].latitude,salt_flux_avg_2005on[0,:,:].longitude])

In [6]:
#water flux from fafmip
f='/scratch/abf376/FAFMIP_wfo_v2.nc' #this is the first 50 years
file2read = netCDF4.Dataset(f,'r')
#print(file2read.variables)
wfo = xr.open_dataset(f)['water_flux_into_sea_water']
wfo=wfo.where(wfo<1E19)


area_wfo=area_grid(latitudes=np.array(wfo.latitude),longitudes=wfo.longitude)
area_wfo=xr.DataArray(area,dims=["latitude","longitude"],coords=[wfo.latitude,wfo.longitude])

In [7]:
#regrid wfo
ds_out = xe.util.grid_global(1, 1)
regridder_wfo= xe.Regridder(wfo, ds_out, "bilinear",periodic=True)
wfo = regridder_wfo(wfo)
wfo=wfo.rename({'y': 'latitude','x': 'longitude'})
wfo=wfo.assign_coords(latitude=wfo.lat[:,0],longitude=wfo.lon[0,:])

/ext3/miniconda3/lib/python3.9/site-packages/xarray/core/dataarray.py:780: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return key in self.data


In [8]:
#now we want to compute the projection of cesm fluxes onto the fafmip perturbation field

y=np.array(np.reshape(wfo.mean('time')*area.where(area.latitude<65),(1,180*360))) #reshape the perturbation field to be a vector

proj=np.empty(45)
err=np.empty(45)
for i in range(0,45):
    x=np.array(np.reshape((salt_flux_avg_2006to2055[(i+5)*12:(i+5)*12,:,:].mean('time'))*area.where(area.latitude<65),(1,180*360)))
    proj[i]=np.nansum(x*y)/(np.nansum(y*y))
    err[i]=(np.nansum(np.abs(proj[i]*y)))/(np.nansum(np.abs(x-proj[i]*y))) #portion of cesm salt flux explained by projection divided by proportion of salt flux explained by rejection. come back to this

In [9]:
# do block bootstrapping to deal with effect of natural variability 

np.random.seed(0)

change=np.empty([3000])
trend=np.empty([45])
p=scipy.stats.linregress(np.linspace(0,44,45), y=proj, alternative='two-sided')
trend=p.intercept+p.slope*np.linspace(0,44,45)

from recombinator.block_bootstrap import circular_block_bootstrap

# number of replications for bootstraps (number of resampled time-series to generate)
B = 3000

y_star_cb \
    = circular_block_bootstrap(proj-trend, 
                               block_length=2, 
                               replications=B, replace=True)
bootstrap=[]
for i in range(0,B):
    bootstrap.append(trend+y_star_cb[i,:])

for i in range(0,B):
    change[i]=(bootstrap[i][40:45].mean()-bootstrap[i][0:5].mean())
print(change.mean())
print(change.std())

5.949971794685712e-01
0.06445565332414613


In [11]:
#Find the change in the pattern strength
z1=np.empty(50)
for i in range(0,50):
    z1[i]=np.nansum(np.array(np.reshape((np.abs(salt_flux_avg_2006to2055[i*12:(i+1)*12,:,:].mean('time')-salt_flux_avg_2006to2055[0*12:(5)*12,:,:].mean('time')))*area.where(area.latitude<65),(1,180*360))))

In [138]:
np.random.seed(0)

changez=np.empty([500])
trend=np.empty([45])
p=scipy.stats.linregress(np.linspace(0,44,45), y=z1[5:50], alternative='two-sided')
trend=p.intercept+p.slope*np.linspace(0,44,45)

from recombinator.block_bootstrap import circular_block_bootstrap

# number of replications for bootstraps (number of resampled time-series to generate)
B = 500

y_star_cb \
    = circular_block_bootstrap(z1[5:50]-trend, 
                               block_length=2, 
                               replications=B, replace=True)
bootstrap=[]
for i in range(0,B):
    bootstrap.append(trend+y_star_cb[i,:])

for i in range(0,B):
    changez[i]=(bootstrap[i][42:45].mean()-bootstrap[i][0:3].mean())

print(changez.mean())
print(changez.std())

736260754.5442837
87605581.97162563


In [14]:
z=np.array(np.reshape((salt_flux_avg_2006to2055[45*12:(50)*12,:,:].mean('time')-salt_flux_avg_2006to2055[5*12:(10)*12,:,:].mean('time'))*area.where(area.latitude<65),(1,180*360)))
np.sqrt(np.nansum((change.mean()*y)**2))/np.sqrt(np.nansum((z)**2)) #near the end of the timeseries, the projection is about half the length of the total

0.5378149853597608

In [47]:
mask = ~np.isnan(change.mean()*y) & ~np.isnan(s)
p=scipy.stats.linregress((change.mean()*y)[mask], s[mask], alternative='two-sided')
print(p)

LinregressResult(slope=1.312149732294195, intercept=-1321.8421978339602, rvalue=0.5761616913882781, pvalue=0.0, stderr=0.00998101518200372, intercept_stderr=237.26210064902753)


In [130]:
l=s
l=np.where(y<0,l,np.abs(l))
l=np.where(y>0,l,-np.abs(l))

mask = ~np.isnan(l) & ~np.isnan(s)
p=scipy.stats.linregress(l[mask], s[mask], alternative='two-sided')
print(p)

LinregressResult(slope=0.7084016093551855, intercept=-2250.817179072021, rvalue=0.7089943027702471, pvalue=0.0, stderr=0.003761169010840046, intercept_stderr=206.9531861217974)


Now the integration way

In [103]:
EP_pattern_wfo=np.empty(50)
for i in range(0,50):
    EP_pattern_wfo[i]=np.abs((((salt_flux_avg_2006to2055[i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))>0))*area.where(area.latitude<65)).sum())+np.abs((((salt_flux_avg_2006to2055[i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))<0))*area.where(area.latitude<65)).sum())

In [104]:
np.random.seed(0)
A=((np.abs(wfo.mean('time')))*area.where(area.latitude<65)).sum() #strength of wfo in this way, 9.99660706e+08 if calculated with regridded or 1.08156723e+09 if not calculated with regridded


change2=np.empty([3000])
trend=np.empty([45])
p=scipy.stats.linregress(np.linspace(0,44,45), y=EP_pattern_wfo[5:50], alternative='two-sided')
trend=p.intercept+p.slope*np.linspace(0,44,45)

from recombinator.block_bootstrap import circular_block_bootstrap

# number of replications for bootstraps (number of resampled time-series to generate)
B = 3000

y_star_cb \
    = circular_block_bootstrap(EP_pattern_wfo[5:50]-trend, 
                               block_length=2, 
                               replications=B, replace=True)
bootstrap=[]
for i in range(0,B):
    bootstrap.append(trend+y_star_cb[i,:])

for i in range(0,B):
    change2[i]=(bootstrap[i][42:45].mean()-bootstrap[i][0:3].mean())/A

print(change2.mean())
print(change2.std())

0.7697617409610814
0.06405399721172099


## Now, we need to do the same for each individual member. First, the projection technique

In [12]:
# a list of all ensemble members
for i in range(0,34):
    regridded_2005on[i]=regridded_2005on[i].rename({'y': 'latitude','x': 'longitude'})
    regridded_2005on[i]=regridded_2005on[i].assign_coords(latitude=salt_flux_avg_2005on.latitude,longitude=salt_flux_avg_2005on.longitude)
    
salt_avg_list=[]
for i in range(0,34):
    salt_avg_list.append(regridded_2005on[i])
    
climatological_salt_flux_list=[]
for i in range(0,34):
    climatological_salt_flux_list.append(salt_avg_list[i][0:12*50,:,:].mean('time')) #this is 1920 to 1975
    salt_avg_list[i]=salt_avg_list[i][0:12*50,:,:]

In [13]:
proj_list=np.empty([34,50])
for j in range(0,34):
    for i in range(0,50):
        x=np.reshape(salt_avg_list[j][i*12:(i+1)*12,:,:].mean('time')*area.where(area.latitude<65),(1,180*360))
        proj_list[j,i]=np.nansum(x*y)/(np.nansum(y*y))

In [14]:
pval_store=np.empty(34)
np.random.seed(0)
change=np.empty([3000,34])
for j in range(0,34):
    trend=np.empty([45])
    p=scipy.stats.linregress(np.linspace(0,44,45), y=proj_list[j,5:50], alternative='two-sided')
    trend=p.intercept+p.slope*np.linspace(0,44,45)
    pval_store[j]=p.pvalue

    from recombinator.block_bootstrap import circular_block_bootstrap

    # number of replications for bootstraps (number of resampled time-series to generate)
    B = 3000

    y_star_cb \
        = circular_block_bootstrap(proj_list[j,5:50]-trend, 
                                   block_length=2, 
                                   replications=B, replace=True)
    bootstrap=[]
    for i in range(0,B):
        bootstrap.append(trend+y_star_cb[i,:])

    for i in range(0,B):
        change[i,j]=(bootstrap[i][40:45].mean()-bootstrap[i][0:5].mean())


mean_boot=np.empty([34])
std_boot=np.empty([34])
for j in range(0,34):
    mean_boot[j]=change[:,j].mean()
    std_boot[j]=change[:,j].std()

In [15]:
import pickle
with open("freshwater_fluxes_projection_mean_boot_2011to2055_new", "wb") as fp:   #Pickling
    pickle.dump(mean_boot, fp)
with open("freshwater_fluxes_projection_std_boot_2011to2055_new", "wb") as fp:   #Pickling
    pickle.dump(std_boot, fp)
with open("freshwater_fluxes_projection_pval_lin_regress_2011to2055_new", "wb") as fp:   #Pickling
    pickle.dump(pval_store, fp)

ensemble_projection_mean_std=[change_ensemble.mean(),change_ensemble.std()]
with open("freshwater_fluxes_ensemble_projection_mean_std_2011to2055_new", "wb") as fp:   #Pickling
    pickle.dump(ensemble_projection_mean_std, fp)

NameError: name 'change_ensemble' is not defined

## Now, we need to do the same for each individual member. Now the integration technique

In [20]:
EP_pattern_wfo_list=np.empty([34,50])
for j in range(0,34):
    for i in range(0,50):
        EP_pattern_wfo_list[j,i]=np.abs((((salt_avg_list[j][i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))>0))*area.where(area.latitude<65)).sum())+np.abs((((salt_avg_list[j][i*12:(i+1)*12,:,:].mean('time')).where(np.sign(wfo.mean('time'))<0))*area.where(area.latitude<65)).sum())

In [26]:
np.random.seed(0)
A=((np.abs(wfo.mean('time')))*area.where(area.latitude<65)).sum() #strength of wfo in this way, 9.99660706e+08 if calculated with regridded or 1.08156723e+09 if not calculated with regridded

change=np.empty([3000,34])
for j in range(0,34):
    trend=np.empty([45])
    p=scipy.stats.linregress(np.linspace(0,44,45), y=EP_pattern_wfo_list[j,5:50], alternative='two-sided')
    trend=p.intercept+p.slope*np.linspace(0,44,45)

    from recombinator.block_bootstrap import circular_block_bootstrap

    # number of replications for bootstraps (number of resampled time-series to generate)
    B = 3000

    y_star_cb \
        = circular_block_bootstrap(EP_pattern_wfo_list[j,5:50]-trend, 
                                   block_length=2, 
                                   replications=B, replace=True)
    bootstrap=[]
    for i in range(0,B):
        bootstrap.append(trend+y_star_cb[i,:])

    for i in range(0,B):
        change[i,j]=(bootstrap[i][42:45].mean()-bootstrap[i][0:3].mean())/A

mean_boot_m2=np.empty([34])
std_boot_m2=np.empty([34])
for j in range(0,34):
    mean_boot_m2[j]=change[:,j].mean()
    std_boot_m2[j]=change[:,j].std()

In [27]:
import pickle
with open("freshwater_fluxes_integration_mean_boot_2011to2055", "wb") as fp:   #Pickling
    pickle.dump(mean_boot_m2, fp)
with open("freshwater_fluxes_integration_std_boot_2011to2055", "wb") as fp:   #Pickling
    pickle.dump(std_boot_m2, fp)
with open("freshwater_fluxes_integration_pval_lin_regress_2011to2055", "wb") as fp:   #Pickling
    pickle.dump(pval_store, fp)

ensemble_projection_mean_std=[change2.mean(),change2.std()]
with open("freshwater_fluxes_ensemble_integration_mean_std_2011to2055", "wb") as fp:   #Pickling
    pickle.dump(ensemble_projection_mean_std, fp)